In [2]:
import pandas as pd

In [3]:
file_path_episodes = "../exploration/BIG_DF_ML.ipynb"

db01 = pd.read_csv("../gitignore/title_basics_traite.csv")
db02 = pd.read_csv("../gitignore/title_ratings_final.tsv", sep="\t")
db03 = pd.read_csv("../gitignore/title.akas_final.tsv", sep="\t")
db04 = pd.read_csv("../gitignore/tmdb_ml_final.csv")
db05 = pd.read_csv("../gitignore/data_bechdel.csv")
db07 = pd.read_csv("../gitignore/name.basics.tsv", sep="\t") 
db08 = pd.read_csv("../gitignore/title.crew.tsv", sep="\t") 

In [4]:
db07[db07['nconst'] == 'nm0000318']

In [5]:
db08[db08['tconst'] == 'tt0107688']

In [6]:
dbmerge_1 = pd.merge(db01, db02, right_on='title_ratings_tconst', left_on='tconst', how='left') #Title Basics + Title Ratings
dbmerge_2 = pd.merge(dbmerge_1, db03, left_on='tconst', right_on='titleId', how='left') # + Title Akas
dbmerge_3 = pd.merge(dbmerge_2, db04, left_on='tconst', right_on='tmdb_imdb_id', how='left') # + TMDB Full
dbmerge_4 = pd.merge(dbmerge_3, db05, left_on='tconst', right_on='imdbid', how='left') # + Bechdel
dbmerge_4 = pd.merge(dbmerge_4, db08, left_on='tconst', right_on='tconst', how='left') # + Title Crew
dbmerge_4 = pd.merge(dbmerge_4, db07, left_on='directors', right_on='nconst', how='left') # + Name Basics

In [7]:
dbmerge_4.info()

In [8]:
#dbmerge_4.to_csv("../gitignore/BIG_DF_ML.csv", index=False)

In [9]:
dbmerge_4['movie'].value_counts()

In [10]:
BIG_DF_ML = dbmerge_4.drop(columns=[
    'titleType',
    'genres', 
    'decade', 
    'Adult',
    'Short',
    'movie',
    'tmdb_Comedy',
    'tmdb_Adventure',
    'tmdb_Drama',
    'tmdb_Crime',
    'tmdb_Action',
    'tmdb_Documentary',
    'tmdb_Animation',
    'tmdb_Mystery',
    'tmdb_Horror',
    'tmdb_Western',
    'tmdb_Science Fiction',
    'tmdb_Thriller',
    'tmdb_Romance',
    'tmdb_Fantasy',
    'tmdb_Family',
    'tmdb_History',
    'tmdb_Music',
    'tmdb_War', 
    'ordering',
    'region',
    'language',
    'types',
    'attributes',
    'isOriginalTitle',
    'birthYear',
    'deathYear',
    'primaryProfession',
    'knownForTitles',
    'directors',
    'writers'
])

In [11]:
BIG_DF_ML.loc[(dbmerge_4['title_ratings_averageRating'].isnull())&(dbmerge_4['tmdb_vote_average'].isnull()), ['title_ratings_tconst', 'title_ratings_averageRating', 'tmdb_vote_average']]

In [12]:
BIG_DF_ML3 = BIG_DF_ML.dropna(subset=['title_ratings_averageRating','tmdb_vote_average'],how='all')

In [13]:
def moyenne_ponderee(ligne):

    # Si 'title_ratings_averageRating' est NaN, on ne prend que 'tmdb_vote_average', et vice versa

    if pd.isna(ligne['title_ratings_averageRating']) and not pd.isna(ligne['tmdb_vote_average']):
        return ligne['tmdb_vote_average']  # Si title_ratings_averageRating est vide, prendre tmdb_vote_average

    elif pd.isna(ligne['tmdb_vote_average']) and not pd.isna(ligne['title_ratings_averageRating']):
        return ligne['title_ratings_averageRating']  # Si tmdb_vote_average est vide, prendre title_ratings_averageRating

    elif not pd.isna(ligne['title_ratings_averageRating']) and not pd.isna(ligne['tmdb_vote_average']):
        # Si les deux colonnes ont des valeurs, calculer la moyenne pondérée
        return (ligne['title_ratings_averageRating'] * ligne['title_ratings_numVotes'] + ligne['tmdb_vote_average'] * ligne['tmdb_vote_count']) / (ligne['title_ratings_numVotes'] + ligne['tmdb_vote_count'])  # Moyenne simple, à ajuster si besoin !
    else:
        return np.nan  # Si les deux sont NaN, retourner NaN


BIG_DF_ML3['notes'] = BIG_DF_ML3.apply(moyenne_ponderee, axis=1)

In [14]:
BIG_DF_ML4 = BIG_DF_ML3.drop(['title_ratings_averageRating','tmdb_vote_average','title_ratings_tconst','titleId','tmdb_imdb_id','imdbid', 'primaryName'], axis=1)

In [15]:
def startyear(ligne):

    # Si 'start year' est 0, il prend la valeur 'tmbd release date'
    BIG_DF_ML4.loc[(BIG_DF_ML4['startYear'] == 0)&(BIG_DF_ML4['tmdb_release_date'] != 0), 'startYear'] = BIG_DF_ML4['tmdb_release_date']

startyear(BIG_DF_ML4)

In [16]:
def release_date(ligne):

    # Si 'tmdn release date' est 0, il prend la valeur 'start year'
    BIG_DF_ML4.loc[(BIG_DF_ML4['tmdb_release_date'] == 0)&(BIG_DF_ML4['startYear'] != 0), 'tmdb_release_date'] = BIG_DF_ML4['startYear']

release_date(BIG_DF_ML4)

In [17]:
def release_date(ligne):

    # Si 'tmdn release date' est 0, il prend la valeur 'start year'
    BIG_DF_ML4.loc[(BIG_DF_ML4['tmdb_release_date'].isna())&(BIG_DF_ML4['startYear'] != 0), 'tmdb_release_date'] = BIG_DF_ML4['startYear']

release_date(BIG_DF_ML4)

In [18]:
def runtimeMinutes(ligne):

    # Si 'runtimeMinutes' est 0, il prend la valeur 'tmdb_runtime'
    BIG_DF_ML4.loc[(BIG_DF_ML4['runtimeMinutes'] == 0)&(BIG_DF_ML4['tmdb_runtime'] != 0), 'runtimeMinutes'] = BIG_DF_ML4['tmdb_runtime']

runtimeMinutes(BIG_DF_ML4)

In [19]:
def runtimeMinutes(ligne):

    # Si 'tmdb_runtime' est 0, il prend la valeur 'runtimeMinutes'
    BIG_DF_ML4.loc[(BIG_DF_ML4['tmdb_runtime'] == 0)&(BIG_DF_ML4['runtimeMinutes'] != 0), 'tmdb_runtime'] = BIG_DF_ML4['runtimeMinutes']

runtimeMinutes(BIG_DF_ML4)

In [20]:
#On transforme les valeurs 0 en NaN pour les supprimer ensuite
import numpy as np
BIG_DF_ML4[BIG_DF_ML4['startYear']==0] = np.nan

In [21]:
#On transforme les valeurs 0 en NaN pour les supprimer ensuite
BIG_DF_ML4[BIG_DF_ML4['runtimeMinutes']==0] = np.nan

In [22]:
BIG_DF_ML5 = BIG_DF_ML4.drop(columns =['tmdb_runtime','tmdb_release_date','tmdb_original_title','tmdb_title','tmdb_vote_count','tmdb_TV Movie'])

In [23]:
BIG_DF_ML5 = BIG_DF_ML5.dropna(subset=['runtimeMinutes'])

In [24]:
BIG_DF_ML5 = BIG_DF_ML5.dropna(subset=['startYear'])

In [25]:
BIG_DF_ML5 = BIG_DF_ML5.dropna(subset=['title'])

In [26]:
# Pour notre ML, on remplace toutes les valeurs nulles des pays de production par False 
Remplacer = [
    'tmdb_US', 'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
    'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE', 'tmdb_SU',
    'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR', 'tmdb_RU', 'tmdb_DK',
    'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR', 'tmdb_PL', 'tmdb_CH', 'tmdb_XC',
    'tmdb_FI', 'tmdb_NO', 'tmdb_IR', 'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'
]
BIG_DF_ML5[Remplacer] = BIG_DF_ML5[Remplacer].fillna(False)

In [27]:
BIG_DF_ML5['rating'] = BIG_DF_ML5['rating'].fillna(0)

In [28]:
BIG_DF_ML5['title_ratings_numVotes'] = BIG_DF_ML5['title_ratings_numVotes'].fillna(0)

In [29]:
BIG_DF_ML5['tmdb_popularity'] = BIG_DF_ML5['tmdb_popularity'].fillna(0)

In [30]:
BIG_DF_ML5['nconst'] = BIG_DF_ML5['nconst'].fillna("")

In [31]:
BIG_DF_ML5[['Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']] = BIG_DF_ML5[['Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']].astype(bool)

In [32]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

import seaborn as sns
import matplotlib.pyplot as plt

In [33]:
export = "../machine learning/DF_ML.csv.gz"
BIG_DF_ML5.to_csv(export, sep=",", index=False, compression="gzip")

In [34]:
df_ml = BIG_DF_ML5

# Test de ML avec cible sur les notes > 7

In [35]:
df_ml.info()

In [36]:
# Obtenir les fréquences des valeurs uniques dans la colonne 'nconst'
counts = df_ml['nconst'].value_counts()

# Filtrer les valeurs qui apparaissent plus de 20 fois
valid_nconst = counts[counts > 50].index

# Garder uniquement les lignes où 'nconst' est dans valid_nconst
filtered_df = df_ml[df_ml['nconst'].isin(valid_nconst)]

# Résultat
print(filtered_df)

In [37]:
df_ml2 = df_ml[df_ml['startYear']>1920]
df_ml2['nconst'].nunique()

On veut entrainer notre modèle sur tout le dataframe et afficher UNIQUEMENT les k films les plus proches dont les notes sont supérieures à 7.

In [38]:
def recommandation(tconst):

     df_ml = pd.read_csv("../machine learning/DF_ML.csv.gz")

     index = df_ml.index
     df_ml_num = df_ml.select_dtypes('number')
     df_ml_cat = df_ml.select_dtypes(['object', 'category', 'string', 'bool'])

     from sklearn.preprocessing import MinMaxScaler
     SN = MinMaxScaler()
     df_ml_num_SN = pd.DataFrame(SN.fit_transform(df_ml_num), columns=df_ml_num.columns, index=index)

     df_ml_encoded = pd.concat([df_ml_num_SN, df_ml_cat], axis=1)

     #On crée une liste des colonnes à utiliser pour le modèle
     caracteristiques = df_ml_encoded.columns.drop(['tconst', 'nconst', 'title', 'tmdb_popularity', 'title_ratings_numVotes','tmdb_US',
          'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
          'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
          'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
          'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
          'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
          'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'])

     #On sépare notre df en deux groupes, en fonction de la note
     bons_films = df_ml_encoded[df_ml_encoded['notes'] >= 0.7]
     mauvais_films = df_ml_encoded[df_ml_encoded['notes'] < 0.7]

     #On crée notre modèle
     model = NearestNeighbors(n_neighbors=1000, metric='euclidean')
     model.fit(bons_films[caracteristiques])

     #On déclare les caractéristiques du film sélectionné par l'utilisateur
     caract_film = df_ml_encoded[df_ml_encoded['tconst'] == tconst]
     caract_film = caract_film[caracteristiques]
     caract_film

     distances, indices = model.kneighbors(caract_film)

     #On affiche la selection des films en fonction des indices trouvés par le modèle
     if caract_film['notes'].values[0] > 0.7:
          distances = distances[0][1:11]
          indices = indices[0][1:11]
          selection = bons_films.iloc[indices]['tconst']
     else:
          distances = distances[0][0:10]
          indices = indices[0][0:10]
          selection = bons_films.iloc[indices]['tconst']

     return selection


In [39]:
selection = recommandation('tt0099487')
pd.DataFrame(selection)

In [40]:
"""
weights = X_encoded[['Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']].astype(bool)
weights *= 2
X_weighted = pd.concat([weights, X_encoded.drop(columns = ['Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Game-Show', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Talk-Show', 'Thriller', 'War', 'Western'])], axis=1)
"""

# Test ML avec TF_IDF (sans les poids)

In [41]:
tmdb = pd.read_csv("../gitignore/tmdb_full.csv")

In [42]:
tmdb = tmdb[['imdb_id', 'genres', 'overview']]

In [43]:
df_test = pd.merge(left=df_ml, right=tmdb, how='left', left_on='tconst', right_on='imdb_id')

In [44]:
def recommandation2(tconst):

     index = df_test.index
     df_test_num = df_test.select_dtypes('number')
     df_test_cat = df_test.select_dtypes(['object', 'category', 'string', 'bool'])

     from sklearn.preprocessing import MinMaxScaler
     SN = MinMaxScaler()
     df_test_num_SN = pd.DataFrame(SN.fit_transform(df_test_num), columns=df_test_num.columns, index=index)

     df_test_encoded = pd.concat([df_test_num_SN, df_test_cat], axis=1)

     #On crée une liste des colonnes à utiliser pour le modèle
     caracteristiques = df_test_encoded.columns.drop(['tconst', 'nconst', 'title', 'tmdb_popularity', 
                                                      'title_ratings_numVotes', 'imdb_id', 'genres', 'overview', 'tmdb_US',
          'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
          'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
          'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
          'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
          'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
          'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'])

     #On sépare notre df en deux groupes, en fonction de la note
     bons_films = df_test_encoded[df_test_encoded['notes'] >= 0.7]
     mauvais_films = df_test_encoded[df_test_encoded['notes'] < 0.7]

     #On crée notre modèle
     model = NearestNeighbors(n_neighbors=1000, metric='euclidean')
     model.fit(bons_films[caracteristiques])

     #On déclare les caractéristiques du film sélectionné par l'utilisateur
     caract_film = df_test_encoded[df_test_encoded['tconst'] == tconst]
     caract_film = caract_film[caracteristiques]
     caract_film

     distances, indices = model.kneighbors(caract_film)

     #On affiche la selection des films en fonction des indices trouvés par le modèle
     colonnes = ['tconst', 'genres', 'overview', 'nconst']  # Liste à étendre si besoin
     selection = bons_films.iloc[indices[0]][colonnes]
     df_selection = pd.DataFrame(selection).reset_index()

     #On crée un nouveau df à partir des données textuelles pour l'étudier avec TF-IDF
     df_tfidf = df_selection.copy()
     df_tfidf['texte'] = df_selection[colonnes].fillna('').agg(' '.join, axis=1)
     df_tfidf = pd.DataFrame(df_tfidf['texte'], columns=['texte'])

     from sklearn.feature_extraction.text import TfidfVectorizer
     from sklearn.metrics.pairwise import cosine_similarity

     #Limite aux 100 mots les plus présents et supprime les mots contenus dans plus de 80% des lignes
     vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8, max_features=100)
     df_tfidf = vectorizer.fit_transform(df_tfidf['texte'])

     #On calcule la distance entre chaque vecteur
     similarite = cosine_similarity(df_tfidf)

     #On crée une liste de tuple avec un index et le score de similarite par rapport au film cible (index 0 car première ligne)
     film_index = df_selection.index[0]
     similarity_scores = list(enumerate(similarite[film_index]))

     #On trie le résultat par rapport aux scores et par ordre decroissant en ignorant la partie index (key permet de cibler uniquement le score)
     similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

     similar_movies = [df_selection['tconst'].iloc[i[0]] for i in similarity_scores[1:10]]

     return similar_movies


     

In [45]:
selection2 = recommandation2('tt0099487')
selection2

# Test ML avec TF_IDF (avec les poids)

In [61]:
def recommandation2(tconst):

     index = df_test.index
     df_test_num = df_test.select_dtypes('number')
     df_test_cat = df_test.select_dtypes(['object', 'category', 'string', 'bool'])

     from sklearn.preprocessing import MinMaxScaler
     SN = MinMaxScaler()
     df_test_num_SN = pd.DataFrame(SN.fit_transform(df_test_num), columns=df_test_num.columns, index=index)

     df_test_encoded = pd.concat([df_test_num_SN, df_test_cat], axis=1)

     #On crée une liste des colonnes à utiliser pour le modèle
     caracteristiques = df_test_encoded.columns.drop(['tconst', 'nconst', 'title', 'tmdb_popularity', 
                                                      'title_ratings_numVotes', 'imdb_id', 'genres', 'overview', 'tmdb_US',
          'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
          'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
          'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
          'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
          'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
          'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA'])

     #On sépare notre df en deux groupes, en fonction de la note
     bons_films = df_test_encoded[df_test_encoded['notes'] >= 0.7]
     mauvais_films = df_test_encoded[df_test_encoded['notes'] < 0.7]

     #On crée notre modèle
     model = NearestNeighbors(n_neighbors=1000, metric='euclidean')
     model.fit(bons_films[caracteristiques])

     #On déclare les caractéristiques du film sélectionné par l'utilisateur
     caract_film = df_test_encoded[df_test_encoded['tconst'] == tconst]
     caract_film = caract_film[caracteristiques]
     caract_film

     distances, indices = model.kneighbors(caract_film)

     #On affiche la selection des films en fonction des indices trouvés par le modèle
     colonnes = ['tconst', 'genres', 'overview', 'nconst']  # Liste à étendre si besoin
     selection = bons_films.iloc[indices[0]][colonnes]
     df_selection = pd.DataFrame(selection).reset_index()

     #On crée une colonne pour identifier le réalisateur s'il correspond à celui du film cible
     cible_real = df_test.loc[df_test['tconst'] == tconst, 'nconst']
     df_selection['real_identique'] = df_selection['nconst'].apply(
        lambda x: 1 if x in cible_real else 0
     )

     #On définit les poids pour chaque colonne textuelle
     colonnes_poids = {
        'tconst': 1,
        'genres': 1,
        'overview': 1,
        'nconst': 20,
        'real_identique': 100
     }

     #On crée une colonne 'texte' pondérée dynamiquement
     df_tfidf = df_selection.copy()
     df_tfidf['texte'] = df_selection.apply(
     lambda ligne: ' '.join(
          (str(ligne[col]) + ' ') * poids for col, poids in colonnes_poids.items()
     ),
     axis=1
     )

     #On crée le DataFrame final pour TF-IDF
     df_tfidf = pd.DataFrame(df_tfidf['texte'], columns=['texte'])

     from sklearn.feature_extraction.text import TfidfVectorizer
     from sklearn.metrics.pairwise import cosine_similarity

     #Limite aux 100 mots les plus présents et supprime les mots contenus dans plus de 80% des lignes
     vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
     df_tfidf = vectorizer.fit_transform(df_tfidf['texte'])

     #On calcule la distance entre chaque vecteur
     similarite = cosine_similarity(df_tfidf)

     #On crée une liste de tuple avec un index et le score de similarite par rapport au film cible (index 0 car première ligne)
     film_index = df_selection.index[0]
     similarity_scores = list(enumerate(similarite[film_index]))

     #On trie le résultat par rapport aux scores et par ordre decroissant en ignorant la partie index (key permet de cibler uniquement le score)
     similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

     similar_movies = [df_selection['tconst'].iloc[i[0]] for i in similarity_scores[1:10]]

     return similarity_scores

     
     

##### Faire un test avec lemmarizing (garder les mots en 'binômes').

In [96]:
def recommandation2(tconst):
    index = df_test.index
    df_test_num = df_test.select_dtypes('number')
    df_test_cat = df_test.select_dtypes(['object', 'category', 'string', 'bool'])

    # Normalize numerical columns
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    df_test_num_SN = pd.DataFrame(SN.fit_transform(df_test_num), columns=df_test_num.columns, index=index)

    # Encode categorical columns
    from sklearn.preprocessing import LabelEncoder
    df_test_cat_encoded = df_test_cat.copy()
    
    for col in df_test_cat_encoded.columns:
        le = LabelEncoder()
        df_test_cat_encoded[col] = le.fit_transform(df_test_cat_encoded[col].fillna('unknown'))
        

    # Combine numeric and encoded data
    df_test_encoded = pd.concat([df_test_num_SN, df_test_cat_encoded], axis=1)
    df_test_encoded['tconst'] = df_test['tconst']  # Add 'tconst' back if it was dropped

    # List of features
    caracteristiques = df_test_encoded.columns.drop(
        ['tconst', 'title', 'tmdb_popularity', 'title_ratings_numVotes', 'imdb_id', 'genres', 'overview', 'tmdb_US',
          'tmdb_FR', 'tmdb_GB', 'tmdb_DE', 'tmdb_JP', 'tmdb_IN', 'tmdb_IT',
          'tmdb_CA', 'tmdb_ES', 'tmdb_MX', 'tmdb_HK', 'tmdb_BR', 'tmdb_SE',
          'tmdb_SU', 'tmdb_PH', 'tmdb_KR', 'tmdb_AU', 'tmdb_CN', 'tmdb_AR',
          'tmdb_RU', 'tmdb_DK', 'tmdb_NL', 'tmdb_BE', 'tmdb_AT', 'tmdb_TR',
          'tmdb_PL', 'tmdb_CH', 'tmdb_XC', 'tmdb_FI', 'tmdb_NO', 'tmdb_IR',
          'tmdb_XG', 'tmdb_EG', 'tmdb_NG', 'tmdb_ZA']
    )

    # Split into "good" and "bad" films
    bons_films = df_test_encoded[df_test_encoded['notes'] >= 0.7]

    # Check if tconst exists
    if tconst not in df_test_encoded['tconst'].values:
        raise ValueError(f"tconst {tconst} not found in the dataset.")

    # Select features for the requested film
    caract_film = df_test_encoded[df_test_encoded['tconst'] == tconst][caracteristiques]

    # NearestNeighbors model
    from sklearn.neighbors import NearestNeighbors
    model = NearestNeighbors(n_neighbors=1000, metric='euclidean')
    model.fit(bons_films[caracteristiques])

    distances, indices = model.kneighbors(caract_film)

    # Select films found
    colonnes = ['tconst', 'genres', 'overview', 'nconst']
    selection = bons_films.iloc[indices[0]][colonnes]
    df_selection = pd.DataFrame(selection).reset_index()

    # Director weighting
    target_directors = df_test.loc[df_test['tconst'] == tconst, 'nconst'].unique()
    df_selection['director_match'] = df_selection['nconst'].apply(
        lambda x: 1 if x in target_directors else 0
    )

    # Weighted column for text
    colonnes_poids = {
        'tconst': 1,
        'genres': 1,
        'overview': 15,
        'nconst': 1
    }

    df_tfidf = df_selection.copy()
    df_tfidf['texte'] = df_selection.apply(
        lambda ligne: ' '.join(
            (str(ligne[col]) + ' ') * poids for col, poids in colonnes_poids.items()
        ),
        axis=1
    )

    # TF-IDF and cosine similarity
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)
    df_tfidf_matrix = vectorizer.fit_transform(df_tfidf['texte'])

    similarite = cosine_similarity(df_tfidf_matrix)
    similarity_scores = list(enumerate(similarite[0]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    similar_movies = [df_selection['tconst'].iloc[i[0]] for i in similarity_scores[1:10]]

    return similar_movies

In [97]:
selection2 = recommandation2('tt1375666')
selection2

#Pour la prochaine fois : utiliser df_selection pour le TFIDF et ne pas avoir les 300K lignes.

In [ ]:


['tt15398776', 'tt0154506', 'tt0482571', 'tt0278504', 'tt0372784', 'tt5013056', 'tt0816692', 'tt6723592', 'tt0468569']

In [50]:
# Ajouter une colonne distance retrouvée après le KNN
# Ajouter une colonne distance retrouvée avec le TFIDF après un nouveau KNN uniquement sur le TFIDF 
# On fait une moyenne pondérée pour trouver le meilleur classement (plus petite en premier)